# RSA (16 bits)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import binascii
import random

### 轉碼

In [7]:
def string_to_int(string):
    byte = string.encode()               # string to byte
    hexadecimal = binascii.hexlify(byte) # byte to hex
    decimal = int(hexadecimal, 16)       # hex to dec
    return decimal

In [8]:
def int_to_string(int):
    hexadecimal = hex(int)               # dec to hex
    byte = hexadecimal[2:].encode()      # hex to byte (type)
    byte = binascii.unhexlify(byte)      # hex to byte (coding)
    string = byte.decode('utf-8')        # byte to utf-8 (支援中文)
    return string

In [66]:
string = 'Hello World!'
integer = string_to_int(string)
print(integer)
message = int_to_string(integer)
print(message)

22405534230753928650781647905
Hello World!


In [67]:
string = '你好！'
integer = string_to_int(string)
print(integer)
message = int_to_string(integer)
print(message)

4219521822575238823041
你好！


### 質因數分解

In [2]:
def factorize(x):
    is_prime = True
    for i in sp.primerange(0, x ** 0.5):
        if x % i == 0:
            j = x // i
            is_prime = False
            return [i, j]
    if is_prime:
        return [0, x]

In [20]:
def factorize_private_key(private_key):
    private_key_fact_list = list()
    if sp.isprime(private_key) == False: # private key 不是質數，可以分解        
        fact = 1
        while fact != 0:
            fact, private_key = factorize(private_key)    
            if fact != 0:
                private_key_fact_list.append(fact)
        private_key_fact_list.append(private_key)
    else:
        private_key_fact_list.append(private_key)
    return private_key_fact_list

In [68]:
factorize(2 ** 10)

[2, 512]

### 最大公因數

In [14]:
def greatest_common_divisor(a, b):
    if b == 0:
        return a
    else:
        return greatest_common_divisor(b, a % b)

### 歐拉函數

In [13]:
def euler_function(p, q):
    return (p - 1) * (q - 1)

### 加密

In [15]:
def encode(plaintext, public_key, n):
    cyphertext = (plaintext ** public_key) % n    
    return cyphertext

### 解密

In [16]:
def decode(cyphertext, private_key, n):
    plaintext = (cyphertext ** private_key) % n
    return plaintext

In [17]:
def decode2(cyphertext, private_key_fact_list, n):
    for i in private_key_fact_list:
        cyphertext = (cyphertext ** i) % n
    plaintext = cyphertext
    return plaintext

### 產生鑰匙

In [19]:
def generate_key():
    public_key, private_key = 0, 0
    while (public_key == 0) or (private_key == 0):
        prime = list(sp.primerange(10 ** 5, 10 ** 6))
        p, q = random.choices(prime, k=2)
        n = p * q
        o = euler_function(p, q)
        public_key, private_key = factorize(o * 10 + 1)
        gcd = greatest_common_divisor(public_key, o)
        remainder = public_key * private_key % o
        print('gcd: %i' % gcd)
        print('remainder: %i' % remainder)
        if gcd == 1 and remainder == 1:
            print('generate key succes\n')
            break
        else:
            print('generate key fail\n')
    return p, q, public_key, private_key, n

In [20]:
def factorize_private_key(private_key):
    private_key_fact_list = list()
    if sp.isprime(private_key) == False: # private key 不是質數，可以分解        
        fact = 1
        while fact != 0:
            fact, private_key = factorize(private_key)    
            if fact != 0:
                private_key_fact_list.append(fact)
        private_key_fact_list.append(private_key)
    else:
        private_key_fact_list.append(private_key)
    return private_key_fact_list

In [21]:
p, q, public_key, private_key, n = generate_key()
print('p: %i' % p)
print('q: %i' % q)
print('public key: %i' % public_key)
print('private key: %i' % private_key)
print('n: %i' % n)

gcd: 1
remainder: 1
generate key succes

p: 997201
q: 674677
public key: 7
private key: 961124153143
n: 672788579077


In [22]:
private_key_fact_list = factorize_private_key(private_key)
print(private_key_fact_list)

[7, 11, 251, 283, 175723]


In [48]:
plaintext = '呵'

In [49]:
plaintext = string_to_int(plaintext)
print(plaintext)

15045045


In [50]:
cyphertext = encode(plaintext, public_key, n)
print(cyphertext)

609975778579


In [51]:
message = decode2(cyphertext, private_key_fact_list, n)
print(message)

15045045


In [52]:
message = int_to_string(message)
print(message)

呵
